In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Layer, Dense, BatchNormalization, Dropout
from tensorflow.keras.models import Model
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler

# Load dataset
url = "https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data"
columns = [
    "age", "workclass", "fnlwgt", "education", "education-num",
    "marital-status", "occupation", "relationship", "race", "sex",
    "capital-gain", "capital-loss", "hours-per-week", "native-country", "income"
]
df = pd.read_csv(url, names=columns, na_values=" ?", skipinitialspace=True)

# Drop missing values
df.dropna(inplace=True)

# Separate features and target
X = df.drop(columns=["income"])
y = df["income"]

# Encode target variable
le = LabelEncoder()
y = le.fit_transform(y)  # Convert to 0 (<=50K) and 1 (>50K)

# Identify categorical and numerical columns
categorical_cols = X.select_dtypes(include=["object"]).columns.tolist()
numerical_cols = X.select_dtypes(include=["int64", "float64"]).columns.tolist()

# Encode categorical features (one-hot encoding)
X = pd.get_dummies(X, columns=categorical_cols, drop_first=True)

# Standardize numerical columns
scaler = StandardScaler()
X[numerical_cols] = scaler.fit_transform(X[numerical_cols])

# Convert to numpy arrays
X = X.values.astype(np.float32)
y = y.astype(np.int32)

# Split into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Get input dimension
input_dim = X_train.shape[1]

# SparseMax Activation
class SparseMax(Layer):
    def call(self, inputs):
        logits_sorted = tf.sort(inputs, direction="DESCENDING")
        cumulative_sum = tf.cumsum(logits_sorted, axis=-1)
        k = tf.range(1, tf.shape(inputs)[-1] + 1, dtype=tf.float32)
        condition = logits_sorted - (cumulative_sum - 1) / k
        k_max = tf.reduce_sum(tf.cast(condition > 0, tf.float32), axis=-1, keepdims=True)
        threshold = (tf.reduce_sum(logits_sorted * tf.cast(condition > 0, tf.float32), axis=-1, keepdims=True) - 1) / k_max
        return tf.maximum(inputs - threshold, 0)

# Feature Transformer (Fix: Match output_dim to input_dim for residual connection)
class FeatureTransformer(Layer):
    def __init__(self, input_dim):
        super(FeatureTransformer, self).__init__()
        self.fc1 = Dense(input_dim, activation=None)  # Match input_dim
        self.bn1 = BatchNormalization()
        self.fc2 = Dense(input_dim, activation=None)  # Match input_dim
        self.bn2 = BatchNormalization()
        self.dropout = Dropout(0.2)

    def call(self, inputs):
        x = self.fc1(inputs)
        x = self.bn1(x)
        x = tf.nn.relu(x)
        x = self.fc2(x)
        x = self.bn2(x)
        x = tf.nn.relu(x)
        return self.dropout(x) + inputs  # Residual connection now works

# Attentive Transformer (Feature Selection)
class AttentiveTransformer(Layer):
    def __init__(self, input_dim):
        super(AttentiveTransformer, self).__init__()
        self.fc = Dense(input_dim, activation=None)
        self.sparsemax = SparseMax()

    def call(self, inputs, prior):
        x = self.fc(inputs)
        x = self.sparsemax(x * prior)  # Apply prior mask
        return x

# TabNet Model
class TabNet(Model):
    def __init__(self, input_dim, output_dim, num_steps=5, gamma=1.5):
        super(TabNet, self).__init__()
        self.num_steps = num_steps
        self.gamma = gamma

        # Initial shared feature transformer
        self.shared_ft = FeatureTransformer(input_dim)  # Fix: Match input_dim

        # Attentive transformers per step
        self.attentive_trans = [AttentiveTransformer(input_dim) for _ in range(num_steps)]

        # Decision layers per step
        self.decision_layers = [Dense(output_dim, activation="softmax") for _ in range(num_steps)]

    def call(self, inputs):
        batch_size = tf.shape(inputs)[0]
        prior = tf.ones((batch_size, inputs.shape[1]))  # Initial prior

        aggregated_output = 0
        masked_inputs = inputs

        for i in range(self.num_steps):
            transformed_features = self.shared_ft(masked_inputs)
            mask = self.attentive_trans[i](inputs, prior)  # Fix: Corrected input to mask selection
            masked_inputs = mask * inputs  # Apply mask to raw input

            # Aggregate decision outputs using weighted sum
            decision_out = self.decision_layers[i](transformed_features)
            aggregated_output += decision_out / self.num_steps  # Averaging effect

            # Update feature prior (increase importance of selected features)
            prior = self.gamma * (prior - mask)

        return aggregated_output

# Create TabNet Model
output_dim = 2  # Binary classification (<=50K or >50K)
tabnet = TabNet(input_dim, output_dim, num_steps=5)

# Compile Model with AdamW optimizer and lower learning rate
tabnet.compile(
    optimizer=tf.keras.optimizers.AdamW(learning_rate=0.005),
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)

# Train the model
history = tabnet.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=100, batch_size=128)

# Evaluate the model
test_loss, test_acc = tabnet.evaluate(X_test, y_test)
print(f"Test Accuracy: {test_acc:.4f}")

# Save the model
tabnet.save("tabnet_adult_model.keras")

#to save this model into a tf lite model
import tensorflow as tf

# Recompile the model with SGD optimizer
tabnet.compile(
    optimizer=tf.keras.optimizers.SGD(learning_rate=0.01, momentum=0.9),
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)

# Retrain the model using SGD
history = tabnet.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=50, batch_size=128)

# Evaluate the retrained model
test_loss, test_acc = tabnet.evaluate(X_test, y_test)
print(f"Test Accuracy after SGD: {test_acc:.4f}")

# Convert to TensorFlow Lite
converter = tf.lite.TFLiteConverter.from_keras_model(tabnet)
tflite_model = converter.convert()

# Save the TFLite model
with open("tabnet_adult_model.tflite", "wb") as f:
    f.write(tflite_model)

print("Model successfully converted and saved as tabnet_adult_model.tflite")

import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Layer, Dense, BatchNormalization, Dropout
from tensorflow.keras.models import Model
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler

# Load dataset
url = "https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data"
columns = [
    "age", "workclass", "fnlwgt", "education", "education-num",
    "marital-status", "occupation", "relationship", "race", "sex",
    "capital-gain", "capital-loss", "hours-per-week", "native-country", "income"
]
df = pd.read_csv(url, names=columns, na_values=" ?", skipinitialspace=True)

# Drop missing values
df.dropna(inplace=True)

# Separate features and target
X = df.drop(columns=["income"])
y = df["income"]

# Encode target variable
le = LabelEncoder()
y = le.fit_transform(y)  # Convert to 0 (<=50K) and 1 (>50K)

# Identify categorical and numerical columns
categorical_cols = X.select_dtypes(include=["object"]).columns.tolist()
numerical_cols = X.select_dtypes(include=["int64", "float64"]).columns.tolist()

# Encode categorical features (one-hot encoding)
X = pd.get_dummies(X, columns=categorical_cols, drop_first=True)

# Standardize numerical columns
scaler = StandardScaler()
X[numerical_cols] = scaler.fit_transform(X[numerical_cols])

# Convert to numpy arrays
X = X.values.astype(np.float32)
y = y.astype(np.int32)

# Split into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Get input dimension
input_dim = X_train.shape[1]

# SparseMax Activation
class SparseMax(Layer):
    def call(self, inputs):
        logits_sorted = tf.sort(inputs, direction="DESCENDING")
        cumulative_sum = tf.cumsum(logits_sorted, axis=-1)
        k = tf.range(1, tf.shape(inputs)[-1] + 1, dtype=tf.float32)
        condition = logits_sorted - (cumulative_sum - 1) / k
        k_max = tf.reduce_sum(tf.cast(condition > 0, tf.float32), axis=-1, keepdims=True)
        threshold = (tf.reduce_sum(logits_sorted * tf.cast(condition > 0, tf.float32), axis=-1, keepdims=True) - 1) / k_max
        return tf.maximum(inputs - threshold, 0)

# Feature Transformer (Fix: Match output_dim to input_dim for residual connection)
class FeatureTransformer(Layer):
    def __init__(self, input_dim):
        super(FeatureTransformer, self).__init__()
        self.fc1 = Dense(input_dim, activation=None)  # Match input_dim
        self.bn1 = BatchNormalization()
        self.fc2 = Dense(input_dim, activation=None)  # Match input_dim
        self.bn2 = BatchNormalization()
        self.dropout = Dropout(0.2)

    def call(self, inputs):
        x = self.fc1(inputs)
        x = self.bn1(x)
        x = tf.nn.relu(x)
        x = self.fc2(x)
        x = self.bn2(x)
        x = tf.nn.relu(x)
        return self.dropout(x) + inputs  # Residual connection now works

# Attentive Transformer (Feature Selection)
class AttentiveTransformer(Layer):
    def __init__(self, input_dim):
        super(AttentiveTransformer, self).__init__()
        self.fc = Dense(input_dim, activation=None)
        self.sparsemax = SparseMax()

    def call(self, inputs, prior):
        x = self.fc(inputs)
        x = self.sparsemax(x * prior)  # Apply prior mask
        return x

# TabNet Model
class TabNet(Model):
    def __init__(self, input_dim, output_dim, num_steps=5, gamma=1.5):
        super(TabNet, self).__init__()
        self.num_steps = num_steps
        self.gamma = gamma

        # Initial shared feature transformer
        self.shared_ft = FeatureTransformer(input_dim)  # Fix: Match input_dim

        # Attentive transformers per step
        self.attentive_trans = [AttentiveTransformer(input_dim) for _ in range(num_steps)]

        # Decision layers per step
        self.decision_layers = [Dense(output_dim, activation="softmax") for _ in range(num_steps)]

    def call(self, inputs):
        batch_size = tf.shape(inputs)[0]
        prior = tf.ones((batch_size, inputs.shape[1]))  # Initial prior

        aggregated_output = 0
        masked_inputs = inputs

        for i in range(self.num_steps):
            transformed_features = self.shared_ft(masked_inputs)
            mask = self.attentive_trans[i](inputs, prior)  # Fix: Corrected input to mask selection
            masked_inputs = mask * inputs  # Apply mask to raw input

            # Aggregate decision outputs using weighted sum
            decision_out = self.decision_layers[i](transformed_features)
            aggregated_output += decision_out / self.num_steps  # Averaging effect

            # Update feature prior (increase importance of selected features)
            prior = self.gamma * (prior - mask)

        return aggregated_output

# Create TabNet Model
output_dim = 2  # Binary classification (<=50K or >50K)
tabnet = TabNet(input_dim, output_dim, num_steps=5)

# Compile Model with AdamW optimizer and lower learning rate
tabnet.compile(
    optimizer=tf.keras.optimizers.AdamW(learning_rate=0.005),
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)

# Train the model
history = tabnet.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=100, batch_size=128)

# Evaluate the model
test_loss, test_acc = tabnet.evaluate(X_test, y_test)
print(f"Test Accuracy: {test_acc:.4f}")

# Save the model
tabnet.save("tabnet_adult_model.keras")

#to save this model into a tf lite model
import tensorflow as tf

# Recompile the model with SGD optimizer
tabnet.compile(
    optimizer=tf.keras.optimizers.SGD(learning_rate=0.01, momentum=0.9),
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)

# Retrain the model using SGD
history = tabnet.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=50, batch_size=128)

# Evaluate the retrained model
test_loss, test_acc = tabnet.evaluate(X_test, y_test)
print(f"Test Accuracy after SGD: {test_acc:.4f}")

# Convert to TensorFlow Lite
converter = tf.lite.TFLiteConverter.from_keras_model(tabnet)
tflite_model = converter.convert()

# Save the TFLite model
with open("tabnet_adult_model.tflite", "wb") as f:
    f.write(tflite_model)

print("Model successfully converted and saved as tabnet_adult_model.tflite")

import tensorflow as tf
import numpy as np

# Convert the trained model to TensorFlow Lite with Post-Training Quantization
converter = tf.lite.TFLiteConverter.from_keras_model(tabnet)

# Enable float16 quantization (reduces model size while maintaining accuracy)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.target_spec.supported_types = [tf.float16]

# Convert the model
quantized_tflite_model = converter.convert()

# Save the quantized TFLite model
with open("tabnet_adult_model_quantized.tflite", "wb") as f:
    f.write(quantized_tflite_model)

print("Quantized TFLite model successfully saved as tabnet_adult_model_quantized.tflite")

Epoch 1/100


/usr/local/lib/python3.11/dist-packages/keras/src/optimizers/base_optimizer.py:774: UserWarning: Gradients do not exist for variables ['tab_net/attentive_transformer_4/dense_6/kernel', 'tab_net/attentive_transformer_4/dense_6/bias'] when minimizing the loss. If using `model.compile()`, did you forget to provide a `loss` argument?
  warnings.warn(


204/204 ━━━━━━━━━━━━━━━━━━━━ 13s 20ms/step - accuracy: 0.7932 - loss: 0.4169 - val_accuracy: 0.8577 - val_loss: 0.3255
Epoch 2/100
204/204 ━━━━━━━━━━━━━━━━━━━━ 5s 20ms/step - accuracy: 0.8547 - loss: 0.3223 - val_accuracy: 0.8581 - val_loss: 0.3211
Epoch 3/100
204/204 ━━━━━━━━━━━━━━━━━━━━ 4s 19ms/step - accuracy: 0.8565 - loss: 0.3109 - val_accuracy: 0.8566 - val_loss: 0.3158
Epoch 4/100
204/204 ━━━━━━━━━━━━━━━━━━━━ 5s 17ms/step - accuracy: 0.8584 - loss: 0.3089 - val_accuracy: 0.8574 - val_loss: 0.3141
Epoch 5/100
204/204 ━━━━━━━━━━━━━━━━━━━━ 4s 20ms/step - accuracy: 0.8608 - loss: 0.3036 - val_accuracy: 0.8603 - val_loss: 0.3189
Epoch 6/100
204/204 ━━━━━━━━━━━━━━━━━━━━ 4s 16ms/step - accuracy: 0.8608 - loss: 0.2993 - val_accuracy: 0.8606 - val_loss: 0.3155
Epoch 7/100
204/204 ━━━━━━━━━━━━━━━━━━━━ 5s 17ms/step - accuracy: 0.8638 - loss: 0.2966 - val_accuracy: 0.8592 - val_loss: 0.3180
Epoch 8/100
204/204 ━━━━━━━━━━━━━━━━━━━━ 5s 23ms/step - accuracy: 0.8639 - loss: 0.2915 - val_accurac

/usr/local/lib/python3.11/dist-packages/keras/src/optimizers/base_optimizer.py:774: UserWarning: Gradients do not exist for variables ['tab_net_1/attentive_transformer_9/dense_18/kernel', 'tab_net_1/attentive_transformer_9/dense_18/bias'] when minimizing the loss. If using `model.compile()`, did you forget to provide a `loss` argument?
  warnings.warn(


204/204 ━━━━━━━━━━━━━━━━━━━━ 13s 24ms/step - accuracy: 0.7888 - loss: 0.4284 - val_accuracy: 0.8569 - val_loss: 0.3291
Epoch 2/100
204/204 ━━━━━━━━━━━━━━━━━━━━ 4s 18ms/step - accuracy: 0.8544 - loss: 0.3269 - val_accuracy: 0.8552 - val_loss: 0.3213
Epoch 3/100
204/204 ━━━━━━━━━━━━━━━━━━━━ 5s 16ms/step - accuracy: 0.8548 - loss: 0.3156 - val_accuracy: 0.8552 - val_loss: 0.3201
Epoch 4/100
204/204 ━━━━━━━━━━━━━━━━━━━━ 6s 20ms/step - accuracy: 0.8566 - loss: 0.3120 - val_accuracy: 0.8578 - val_loss: 0.3256
Epoch 5/100
204/204 ━━━━━━━━━━━━━━━━━━━━ 3s 16ms/step - accuracy: 0.8597 - loss: 0.3087 - val_accuracy: 0.8548 - val_loss: 0.3215
Epoch 6/100
204/204 ━━━━━━━━━━━━━━━━━━━━ 4s 18ms/step - accuracy: 0.8599 - loss: 0.3027 - val_accuracy: 0.8523 - val_loss: 0.3304
Epoch 7/100
204/204 ━━━━━━━━━━━━━━━━━━━━ 5s 23ms/step - accuracy: 0.8636 - loss: 0.3025 - val_accuracy: 0.8577 - val_loss: 0.3223
Epoch 8/100
204/204 ━━━━━━━━━━━━━━━━━━━━ 4s 16ms/step - accuracy: 0.8607 - loss: 0.2987 - val_accurac

In [2]:
import tensorflow as tf
import numpy as np

# Load the quantized model
interpreter = tf.lite.Interpreter(model_path="tabnet_adult_model_quantized.tflite")
interpreter.allocate_tensors()

# Get input and output details
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Function to make predictions using TFLite model
def predict_tflite(interpreter, X_test):
    predictions = []
    for i in range(len(X_test)):
        input_data = np.expand_dims(X_test[i], axis=0).astype(np.float32)  # Ensure correct dtype
        interpreter.set_tensor(input_details[0]['index'], input_data)
        interpreter.invoke()
        output_data = interpreter.get_tensor(output_details[0]['index'])
        predictions.append(np.argmax(output_data))  # Get class with highest probability
    return np.array(predictions)

# Run inference
y_pred_tflite = predict_tflite(interpreter, X_test)

# Calculate accuracy
accuracy_tflite = np.mean(y_pred_tflite == y_test)
print(f"Post-Training Quantization Model Accuracy: {accuracy_tflite:.4f}")


Post-Training Quantization Model Accuracy: 0.8532


In [3]:
!ls /content/


sample_data		  tabnet_adult_model_quantized.tflite
tabnet_adult_model.keras  tabnet_adult_model.tflite


In [4]:
from google.colab import files
files.download("tabnet_adult_model_quantized.tflite")

from google.colab import files
files.download("tabnet_adult_model.tflite")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [5]:
import os

# Evaluate original model accuracy
test_loss, test_acc = tabnet.evaluate(X_test, y_test)
print(f"Test Accuracy before Post-Training Quantization: {test_acc:.4f}")

# Load the quantized TFLite model and evaluate accuracy
interpreter = tf.lite.Interpreter(model_path="tabnet_adult_model_quantized.tflite")
interpreter.allocate_tensors()

# Get input and output details
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Function to run inference on TFLite model
def evaluate_tflite_model(interpreter, X_test, y_test):
    correct_predictions = 0
    total_samples = len(y_test)

    for i in range(total_samples):
        # Preprocess input
        input_data = np.expand_dims(X_test[i], axis=0).astype(np.float32)

        # Set input tensor
        interpreter.set_tensor(input_details[0]['index'], input_data)

        # Run inference
        interpreter.invoke()

        # Get output tensor
        output_data = interpreter.get_tensor(output_details[0]['index'])
        predicted_label = np.argmax(output_data)

        # Check if prediction is correct
        if predicted_label == y_test[i]:
            correct_predictions += 1

    return correct_predictions / total_samples

# Compute accuracy of quantized TFLite model
quantized_acc = evaluate_tflite_model(interpreter, X_test, y_test)
print(f"Test Accuracy after Post-Training Quantization: {quantized_acc:.4f}")

# Get file sizes
tflite_model_size = os.path.getsize("tabnet_adult_model.tflite") / 1024  # in KB
quantized_tflite_model_size = os.path.getsize("tabnet_adult_model_quantized.tflite") / 1024  # in KB

print(f"Original TFLite Model Size: {tflite_model_size:.2f} KB")
print(f"Quantized TFLite Model Size: {quantized_tflite_model_size:.2f} KB")


204/204 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - accuracy: 0.8546 - loss: 0.4047
Test Accuracy before Post-Training Quantization: 0.8534
Test Accuracy after Post-Training Quantization: 0.8532
Original TFLite Model Size: 265.80 KB
Quantized TFLite Model Size: 148.21 KB


In [9]:
import tensorflow as tf
import numpy as np
import os

# Convert the trained model to TensorFlow Lite with Post-Training Quantization
converter = tf.lite.TFLiteConverter.from_keras_model(tabnet)

# Enable float16 quantization (reduces model size while maintaining accuracy)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.target_spec.supported_types = [tf.float16]

# Convert the model
quantized_tflite_model = converter.convert()

# Save the quantized TFLite model
with open("tabnet_adult_model_quantized.tflite", "wb") as f:
    f.write(quantized_tflite_model)

print("Quantized TFLite model successfully saved as tabnet_adult_model_quantized.tflite")

# Evaluate original model accuracy and loss
test_loss, test_acc = tabnet.evaluate(X_test, y_test)
print(f"Test Accuracy before Post-Training Quantization: {test_acc:.4f}")
print(f"Test Loss before Post-Training Quantization: {test_loss:.4f}")

# Load the quantized TFLite model and evaluate accuracy
interpreter = tf.lite.Interpreter(model_path="tabnet_adult_model_quantized.tflite")
interpreter.allocate_tensors()

# Get input and output details
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Function to run inference on TFLite model
def evaluate_tflite_model(interpreter, X_test, y_test):
    correct_predictions = 0
    total_samples = len(y_test)

    for i in range(total_samples):
        # Preprocess input
        input_data = np.expand_dims(X_test[i], axis=0).astype(np.float32)

        # Set input tensor
        interpreter.set_tensor(input_details[0]['index'], input_data)

        # Run inference
        interpreter.invoke()

        # Get output tensor
        output_data = interpreter.get_tensor(output_details[0]['index'])
        predicted_label = np.argmax(output_data)

        # Check if prediction is correct
        if predicted_label == y_test[i]:
            correct_predictions += 1

    return correct_predictions / total_samples

# Compute accuracy of quantized TFLite model
quantized_acc = evaluate_tflite_model(interpreter, X_test, y_test)
print(f"Test Accuracy after Post-Training Quantization: {quantized_acc:.4f}")

# Get file sizes
tflite_model_size = os.path.getsize("tabnet_adult_model.tflite") / 1024  # in KB
quantized_tflite_model_size = os.path.getsize("tabnet_adult_model_quantized.tflite") / 1024  # in KB

print(f"Original TFLite Model Size: {tflite_model_size:.2f} KB")
print(f"Quantized TFLite Model Size: {quantized_tflite_model_size:.2f} KB")

print("\n===== FINAL RESULTS =====")
print(f"Test Accuracy before Quantization: {test_acc * 100:.2f}%")
print(f"Test Loss before Quantization: {test_loss * 10:.2f}%")
print(f"Test Accuracy after Quantization: {quantized_acc * 100:.2f}%")
print(f"Original TFLite Model Size: {tflite_model_size:.2f} KB")
print(f"Quantized TFLite Model Size: {quantized_tflite_model_size:.2f} KB")


Saved artifact at '/tmp/tmpm6_yc8wg'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 100), dtype=tf.float32, name=None)
Output Type:
  TensorSpec(shape=(None, 2), dtype=tf.float32, name=None)
Captures:
  135567640854352: TensorSpec(shape=(), dtype=tf.resource, name=None)
  135567640841296: TensorSpec(shape=(), dtype=tf.resource, name=None)
  135567638512080: TensorSpec(shape=(), dtype=tf.resource, name=None)
  135567638513040: TensorSpec(shape=(), dtype=tf.resource, name=None)
  135567640841872: TensorSpec(shape=(), dtype=tf.resource, name=None)
  135567640843024: TensorSpec(shape=(), dtype=tf.resource, name=None)
  135567638512464: TensorSpec(shape=(), dtype=tf.resource, name=None)
  135567638511696: TensorSpec(shape=(), dtype=tf.resource, name=None)
  135567638512848: TensorSpec(shape=(), dtype=tf.resource, name=None)
  135567638514192: TensorSpec(shape=(), dtype=tf.resource, name=None)
  135567638514000: TensorSpec(shap